In [ ]:
import numpy as np
import torch

import sys
if "/home/yskim/projects/sparse-bfp" not in sys.path:
    sys.path.append("/home/yskim/projects/sparse-bfp")
from util.bfp.bfp_wrapper import BfpWrapper

In [ ]:
def get_np(path):
    with open(path, "rb") as f:
        return np.load(f, allow_pickle=False)

In [ ]:
def get_bfp_sparsity(bfp_path, layer_name, layer_id, phase):
    # lhs_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-lhs-E-bfp.npy")
    # lhs_ori_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-lhs-ori-E-bfp.npy")
    # lhs_E_offsets = lhs_E - lhs_ori_E
    # lhs_E_zero_indices = np.where(lhs_E_offsets >= 8)

    lhs_M = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-lhs-M-bfp.npy")
    lhs_M_nonzeros = np.count_nonzero(lhs_M, axis=1)
    lhs_cols = lhs_M.shape[1]
    lhs_M_zeros = lhs_cols - lhs_M_nonzeros
    lhs_sparsity = lhs_M_zeros / lhs_cols * 100.0

    # rhs_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-rhs-E-bfp.npy")
    # rhs_ori_E = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-rhs-ori-E-bfp.npy")
    # rhs_E_offsets = rhs_E - rhs_ori_E
    # rhs_E_zero_indices = np.where(rhs_E_offsets >= 8)


    rhs_M = get_np(f"{bfp_path}/{layer_name}-{layer_id}-{phase}-rhs-M-bfp.npy")
    rhs_M_nonzeros = np.count_nonzero(rhs_M, axis=1)
    rhs_cols = rhs_M.shape[1]
    rhs_M_zeros = rhs_cols - rhs_M_nonzeros
    rhs_sparsity = rhs_M_zeros / rhs_cols * 100.0

    return lhs_sparsity, rhs_sparsity, lhs_M.shape, rhs_M.shape # , lhs_E_zero_indices, rhs_E_zero_indices

def get_fp_sparsity(fp_path, layer_name, layer_id, phase):
    lhs = get_np(f"{fp_path}/{layer_name}-{layer_id}-{phase}-lhs-fp.npy")
    lhs_M_nonzeros = np.count_nonzero(lhs, axis=1)
    lhs_cols = lhs.shape[1]
    lhs_M_zeros = lhs_cols - lhs_M_nonzeros
    lhs_sparsity = lhs_M_zeros / lhs_cols * 100.0

    rhs = get_np(f"{fp_path}/{layer_name}-{layer_id}-{phase}-rhs-fp.npy").transpose()
    rhs_M_nonzeros = np.count_nonzero(rhs, axis=1)
    rhs_cols = rhs.shape[1]
    rhs_M_zeros = rhs_cols - rhs_M_nonzeros
    rhs_sparsity = rhs_M_zeros / rhs_cols * 100.0

    return lhs_sparsity, rhs_sparsity # , lhs, rhs

In [ ]:
def get_sparsity(data_root, conv2d_num, linear_num, iterations, phase, M_bit):
    fp_root = f"{data_root}/fp"
    bfp_root = f"{data_root}/bfp-m{M_bit}"

    conv2d_result = {
        "fp" : {
            "lhs": [],
            "rhs": []
        },
        "bfp": {
            "lhs": [],
            "rhs": []
        }
    }

    linear_result = {
        "fp" : {
            "lhs": [],
            "rhs": []
        },
        "bfp": {
            "lhs": [],
            "rhs": []
        }
    }

    for i in range(iterations):
        print(f"[{phase}] iter {i}th")
        fp_path = f"{fp_root}/iter-{i}"
        bfp_path = f"{bfp_root}/iter-{i}"

        for conv2d_id in range(1 if phase == "grad-a" else 0, conv2d_num):
            bfp_lhs_sparsity, bfp_rhs_sparsity, bfp_lhs_M_shape, bfp_rhs_M_shape = get_bfp_sparsity(bfp_path, "conv2d", conv2d_id, phase)
            bfp_lhs_sparsity, bfp_rhs_sparsity = np.mean(bfp_lhs_sparsity), np.mean(bfp_rhs_sparsity)
            conv2d_result["bfp"]["lhs"].append(bfp_lhs_sparsity)
            conv2d_result["bfp"]["rhs"].append(bfp_rhs_sparsity)

            fp_lhs_sparsity, fp_rhs_sparsity = get_fp_sparsity(fp_path, "conv2d", conv2d_id, phase)
            fp_lhs_sparsity, fp_rhs_sparsity = np.mean(fp_lhs_sparsity), np.mean(fp_rhs_sparsity)
            conv2d_result["fp"]["lhs"].append(fp_lhs_sparsity)
            conv2d_result["fp"]["rhs"].append(fp_rhs_sparsity)

            # fp_lhs_filtered = fp_lhs[lhs_E_zero_indices]
            # fp_rhs_filtered = fp_rhs[rhs_E_zero_indices]

            # print(f"{fp_lhs_filtered[0]}, {fp_lhs_filtered[0]}")

            print(f"conv2d {conv2d_id:>3}th: [{bfp_lhs_M_shape[0]:>5}, {bfp_lhs_M_shape[1]:>5}] x [{bfp_rhs_M_shape[0]:>5} x {bfp_rhs_M_shape[1]:>5}], "
                #   f"[fp sparsity] lhs: {fp_lhs_sparsity:.2f}%, rhs: {fp_rhs_sparsity:.2f}%, [bfp sparsity] lhs {bfp_lhs_sparsity:.2f}%, rhs: {bfp_rhs_sparsity:.2f}%")
                f"[lhs sparsity] fp: {fp_lhs_sparsity:.2f}%, bfp: {bfp_lhs_sparsity:.2f}%, [rhs sparsity] fp: {fp_rhs_sparsity:.2f}%, bfp: {bfp_rhs_sparsity:.2f}%")


        for linear_id in range(linear_num):
            bfp_lhs_sparsity, bfp_rhs_sparsity, bfp_lhs_M_shape, bfp_rhs_M_shape = get_bfp_sparsity(bfp_path, "linear", linear_id, phase)
            bfp_lhs_sparsity, bfp_rhs_sparsity = np.mean(bfp_lhs_sparsity), np.mean(bfp_rhs_sparsity)
            linear_result["bfp"]["lhs"].append(bfp_lhs_sparsity)
            linear_result["bfp"]["rhs"].append(bfp_rhs_sparsity)

            fp_lhs_sparsity, fp_rhs_sparsity = get_fp_sparsity(fp_path, "linear", linear_id, phase)
            fp_lhs_sparsity, fp_rhs_sparsity = np.mean(fp_lhs_sparsity), np.mean(fp_rhs_sparsity)
            linear_result["fp"]["lhs"].append(fp_lhs_sparsity)
            linear_result["fp"]["rhs"].append(fp_rhs_sparsity)

            # fp_lhs_filtered = fp_lhs[lhs_E_zero_indices]
            # fp_rhs_filtered = fp_rhs[rhs_E_zero_indices]

            # print(f"{fp_lhs_filtered[0]}, {fp_lhs_filtered[0]}")

            print(f"linear {linear_id:>3}th: [{bfp_lhs_M_shape[0]:>5}, {bfp_lhs_M_shape[1]:>5}] x [{bfp_rhs_M_shape[0]:>5} x {bfp_rhs_M_shape[1]:>5}], "
                #   f"[fp sparsity] lhs: {fp_lhs_sparsity:.2f}%, rhs: {fp_rhs_sparsity:.2f}%, [bfp sparsity] lhs {bfp_lhs_sparsity:.2f}%, rhs: {bfp_rhs_sparsity:.2f}%")
                f"[lhs sparsity] fp: {fp_lhs_sparsity:.2f}%, bfp: {bfp_lhs_sparsity:.2f}%, [rhs sparsity] fp: {fp_rhs_sparsity:.2f}%, bfp: {bfp_rhs_sparsity:.2f}%")

    return conv2d_result, linear_result

In [ ]:
data_root = "/home/yskim/projects/sparse-bfp/bfp-iter-log/tv-vgg13_bn-b32-fp-cuda-e30-ori"
conv2d_num = 10
linear_num = 3
iterations = 1


# data_root = "/home/yskim/projects/sparse-bfp/bfp-iter-log/tv-resnet18-b32-fp-cuda-e30"
# conv2d_num = 20
# linear_num = 1
# iterations = 1

M_bit = [2, 4, 6]
phases = ["fwd", "grad-w", "grad-a"]

results = {}

for m_bit in M_bit:
    print(f"M_bit: {m_bit}")
    result = {}
    for phase in phases:
        conv2d_result, linear_result = get_sparsity(data_root, conv2d_num, linear_num, iterations, phase, m_bit)
        result[phase] = {
            "conv2d": conv2d_result,
            "linear": linear_result
        }

    results[m_bit] = result


In [ ]:
import matplotlib.pyplot as plt
import matplotlib

def show_result(result, axs, color, pos, start=None, count=None, is_fp=False):
    phases = ["fwd", "grad-w", "grad-a"]
    sides = ["lhs", "rhs"]
    width = 0.4

    r = 0
    for phase in phases:
        c = 0
        for side in sides:
            ax = axs[r,c]
            # axs[curr_plot_idx].subplot(len(phases), len(sides), curr_plot_idx)
            ax.set_title(f"{side} in {phase.upper()}")
            ax.set_ylim(0,100)
            ax.set_xlabel("Layer index")
            ax.set_ylabel("Sparsity %")

            result_phase = result[phase]
            conv2d_result = result_phase["conv2d"]
            linear_result = result_phase["linear"]

            if start is None or count is None:
                sparsity = (conv2d_result["bfp" if not is_fp else "fp"][side] + linear_result["bfp" if not is_fp else "fp"][side])
            else:
                sparsity = (conv2d_result["bfp" if not is_fp else "fp"][side] + linear_result["bfp" if not is_fp else "fp"][side])[start:start+count]
            ax.xaxis.set_ticks(np.arange(len(sparsity)) * 2, np.arange(len(sparsity)) + start)
            # ax.plot(np.arange(len(sparsity)), sparsity, color=color, marker='o', linestyle='--')
            step = (width / 2.0) * (1.0 if pos < 0 else -1.0)
            ax.bar(np.arange(len(sparsity)) * 2 + (pos * width + step), sparsity, width, color=color)

            # curr_plot_idx += 1
            c += 1
        r += 1

matplotlib.rcParams.update({'font.size': 16})
# fig, axs = plt.subplots(3, 2, figsize=(60, 16))
fig, axs = plt.subplots(3, 2, figsize=(24, 16))

model_name = "VGG13 (ori)"
dataset_name = "CIFAR100"
lr = 0.01

fig.suptitle(f"{model_name} on {dataset_name} (lr={lr})")
# fig.tight_layout()
fig.subplots_adjust(hspace=0.4, wspace=0.3)

start = 0
count = None

show_result(results[6], axs, "lightsteelblue", -2, start, count, is_fp=True)
show_result(results[6], axs, "cornflowerblue", -1, start, count,)
show_result(results[4], axs, "royalblue", 1, start, count,)
show_result(results[2], axs, "midnightblue", 2, start, count,)

plt.legend(labels=["FP32", "BFP 6-bit", "BFP 4-bit", "BFP 2-bit"], bbox_to_anchor=(1, 4.1), ncol=4, loc="upper right")
plt.savefig(f"/home/yskim/projects/sparse-bfp/pictures/{model_name}-{dataset_name}-lr{lr}-start{start}-end{count if count is not None else (conv2d_num + linear_num)}.png")
# show_result(results[6], axs, "lavender", -2, is_fp=True)
# show_result(results[6], axs, "lightsteelblue", -1)
# show_result(results[4], axs, "cornflowerblue", 1)
# show_result(results[2], axs, "royalblue", 2)

# show_result(results[6], axs, "y", 2, is_fp=True)